# Interactive tutorial

Intention of this tutorial, written as jupyter notebook, is to provide a quick interactive introduction how to use `vrp_cli` package published in [pypi registry](https://pypi.org/project/vrp-cli/). This package provides a basic API to solve rich `Vehicle Routing Problems` using [pragmatic format](https://reinterpretcat.github.io/vrp/concepts/pragmatic/index.html).

## Dependencies

This section explains how to get required dependencies on your local environment (tested with Python 3.10).

### Install package dependencies

First, we need to install helper dependencies for modeling an input (a VRP `Problem`) and output (a VRP `Solution`) using strictly typed models:

```bash
pip install pydantic typing dataclasses
```

Next, install the latest version of the library (`1.22.0` as time of writting)
```bash
pip install vrp-cli
```

### Get example models

At the moment, `vrp_cli` package doesn't include models for `pragmatic` format used in rust code natively. This is possible to do, but would require some work (see [#125](https://github.com/reinterpretcat/vrp/issues/125) for tracking the issue). So, the current solution is to prepare them in a semi-automated way with help of external libraries such as `pydantic`. 

Let's start by importing the models from a local folder (or from [remote](https://github.com/reinterpretcat/vrp/tree/master/examples/python-interop)):

In [1]:
import pragmatic_types as prg
import config_types as cfg

Please note, that imported python models are not yet fully complete, but this can be tweaked when necessary to have feature parity with the solver features.

## Modeling a VRP problem

Next step is to dig into the core concepts which are used to model VRP problem. 

Here, each problem consists of a `Plan` and a `Fleet`. A `Plan` specifies the work which should be done defined as a list of one or more `Job`s. A `Fleet` specifies actual workers which are responsible for jobs to be completed. It consists of one or more `VehicleType`s. 

Let's look at jobs and vehicle types a bit closer.

### Job

Each job is represented by:

- unique `id`
- one or multiple `JobTask`s inside `pickups` or `deliveries` properties (actually, more is supported, but it is out of scope for the tutorial)

Essentially, each job task specifies a unit of work to be done at specific place and, optionally, time. Depending on how job tasks are specified, we can identify three general job types:

- `pickup` job
- `delivery` job
- `multi` job

#### Pickup job

When a job has only one pickup defined as a job task, then it is considered as a `pickup` job. Here is a definition of a trivial pickup job:

In [2]:
pickup_job = prg.Job(
    id='pickup_job',
    pickups=[
        prg.JobTask(
            places=[
                prg.JobPlace(
                    location=prg.Location(lat=52.5225, lng=13.4095),
                    duration=240,
                    times=[['2024-07-04T10:00:00Z', '2024-07-04T16:00:00Z']]
                )
            ],
            demand=[1]
        )
    ]
)

Logically, the vehicle picks some package at pickup locations (here, it is (52.5225, 13.4095)), which leads to capacity growth according to job.pickups.demand value (respectively, 1 abstract unit), and brings it till the end of the tour.

Here, additionally it has:
- duration (service time): 240 seconds
- a single time window from `2024-07-04T10:00:00Z` to `2024-07-04T16:00:00Z`

Please note, that we specified just one `JobPlace`, but it is possible to specify many. When many job places are specified, each of them is considered as an alternative and only one will be selected depending on efficiency (typically, specified by objective function). See [documentation](https://reinterpretcat.github.io/vrp/concepts/pragmatic/problem/jobs.html#places) for more details.

#### Delivery job

When a job has only one delivery defined as a job task, then it is considered as a `delivery` job. Here is a definition of a such trivial delivery job:


In [3]:
delivery_job = prg.Job(
    id='delivery_job',
    deliveries=[
        prg.JobTask(
            places=[
                prg.JobPlace(
                    location=prg.Location(lat=52.52599, lng=13.45413),
                    duration=300,
                    times=[['2024-07-04T09:00:00Z', '2024-07-04T18:00:00Z']]
                ),
            ],
            demand=[1]
        )
    ]
)

Logically, the vehicle picks some parcel at the start stop, which leads to initial capacity growth, and brings it to job's locations,
where capacity is decreased based on `job.deliveries.demand` values.

Besides `deliveries` property name, the definition looks exactly the same as for pickup job.

#### Multi-job

A multi job is a job with both `job.pickups` and `job.deliveries` properties specified. The simplest example is when only one pickup and one delivery is specified:

In [4]:
multi_job = prg.Job(
    id="multi_job",
    pickups=[
        prg.JobTask(
            places=[
                prg.JobPlace(
                    location=prg.Location(lat=52.5225, lng=13.4095),
                    duration=300
                )
            ],
            demand=[1]
        )
    ],
    deliveries=[
        prg.JobTask(
            places=[
                prg.JobPlace(
                    location=prg.Location(lat=52.5165, lng=13.3808),
                    duration=300
                ),
            ],
            demand=[1]
        )
    ]
)


Logically, the vehicle picks some parcel at pickup location and brings it to delivery location. Such jobs have the following rules:

- all pickup/delivery tasks should be done or none of them
- assignment order is not defined except all pickups should be assigned before any of deliveries
- sum of pickup demand should be equal to sum of delivery demand

### Vehicle

Once jobs are specified, the next step is to define vehicles used. Each vehicle belongs to a specific `VehicleType`, so we need to specify one:

In [5]:
vehicle_type = prg.VehicleType(
    typeId='vehicle',
    vehicleIds=['vehicle_1'],
    profile=prg.VehicleProfile(matrix='normal_car'),
    costs=prg.VehicleCosts(fixed=22, distance=0.0002, time=0.005),
    shifts=[
        prg.VehicleShift(
            start=prg.VehicleShiftStart(
                earliest="2024-07-04T09:00:00Z",
                location=prg.Location(lat=52.5316, lng=13.3884),
            ),
            end=prg.VehicleShiftEnd(
                latest="2024-07-04T18:00:00Z",
                location=prg.Location(lat=52.5316, lng=13.3884),
            )
        )
    ],
    capacity=[10]
)

Quick summary of the properties above:
- `typeId`: a unique vehicle type id. Think about it as an class type in programming languages (Java, C#, C++, etc.).
- `vehicleIds`: a list of unique vehicle ids. Each type can have more than one vehicle ids. This is similar to concrete instances of a specific class type
- `profile`: a reference to routing matrix profile with name `normal_car`. Will be covered in next session
- `costs`: specifies distance/duration costs per meter/second and a fixed cost, applied once per tour (see below).
    Depending on your goal of optimization, you can put a different ratio between distance/duration costs (or use different objectives which is more [advanced topic](https://reinterpretcat.github.io/vrp/concepts/pragmatic/problem/objectives.html))
- `shifts`: specifies where and when vehicle starts and ends. End is optional to model Open VRP variant
- `capacity`: specifies vehicle's capacity to handle job's demand

### Routing data

The last step in modeling VRP problem is to define how routing data, distance/duration between any two locations, is supplied to the solver. The solver uses concept of `RoutingProfile` which is used as a reference to supplied routing data.


#### Routing profile

For our example, the routing profile can be defined the following way:

In [6]:
profile = prg.RoutingProfile(name='normal_car')

### Routing matrix

A routing matrix constains information about duration and distance between any of two locations. Both durations and distances are mapped to the list of unique locations generated from the problem definition. In this list, locations are specified in the order they defined. For example, if you have two jobs with locations A and B, one vehicle type with depot location C, then you have the following location list: A,B,C. It corresponds to the matrix (durations or distances):


|    |    |    |
|----|----|----|
|  0 | AB | AC |
| BA |  0 | BC |
| CA | CB |  0 |

where
- `0`: zero duration or distance
- `XY`: distance or duration from X location to Y

As single dimensional array it looks like:

    [0,AB,AC,BA,0,BC,CA,CB,0]

As we 4 unique locations in our problem definition (note that `pickup_job` and `multi_job` are sharing the same location), we need to have a matrix which contains duration and distance arrays with 16 items in each:

In [7]:
matrix = prg.RoutingMatrix(
    profile='normal_car',
    durations=[0, 609, 981, 906, 813, 0, 371, 590, 1055, 514, 0, 439, 948, 511, 463, 0],
    distances=[0, 3840, 5994, 5333, 4696, 0, 2154, 3226, 5763, 2674, 0, 2145, 5112, 2470, 2152, 0]
)

Please refer to [routing format](https://reinterpretcat.github.io/vrp/concepts/pragmatic/routing/format.html) documentation for more details.

**NOTE**: alternatively, the solver supports distance/duration approximation with help of Haversine formula in case of no routing matrix available. This could be useful for algorithm testing and/or prototyping.

## Configuration

The last step, before we can run the solver, is to configure solver how long it should be run, what kind of logging details we want to have, etc. Such settings are passed via `Config`: 

In [8]:
config = cfg.Config(
    termination=cfg.Termination(
        maxTime=1,
        maxGenerations=3000
    )
)

Here, we specify maximum running time (1 second) and maximum amount of generations (or iterations, 3000). Actual values depend on problem size, variant and your hardware.

## Run the solver

Now, it's time to combine all pieces together. First, let's create a `Problem` definition using jobs, vehicle and profile definitions from above:

In [9]:
problem = prg.Problem(
    plan=prg.Plan(
        jobs=[delivery_job, pickup_job, multi_job]
    ),
    fleet=prg.Fleet(
        vehicles=[vehicle_type],
        profiles=[profile]
    )
)

Now, we're ready to run the solver using vrp-cli's `solve_pragmatic` function which requires models serialized as json strings (a subject for improvement, see [this ticket](https://github.com/reinterpretcat/vrp/issues/125)). So, we import necessary modules and run the solver:

In [10]:
import vrp_cli
import json
from pydantic.json import pydantic_encoder

solution_json = vrp_cli.solve_pragmatic(
    problem=json.dumps(problem, default=pydantic_encoder),
    matrices=[json.dumps(matrix, default=pydantic_encoder)],
    config=json.dumps(config, default=pydantic_encoder),
)

configured to use max-generations: 3000
configured to use max-time: 1s
configured to use custom heuristic
total jobs: 3, actors: 1
preparing initial solution(-s)
[0s] created initial solution in 0ms, fitness: (0.000, 1.000, 47.551)
[0s] created initial solution in 0ms, fitness: (0.000, 1.000, 47.551)
[0s] created initial solution in 0ms, fitness: (0.000, 1.000, 47.551)
[0s] created initial solution in 0ms, fitness: (0.000, 1.000, 47.551)
created initial population in 1ms
[0s] generation 0 took 0ms, median: 0ms fitness: (0.000, 1.000, 47.551)
[0s] population state (phase: initial, speed: 0.00 gen/sec, improvement ratio: 0.000:0.000):
	rank: 0, fitness: (0.000, 1.000, 47.551), difference: 0.000%
[0s] generation 100 took 0ms, median: 0ms fitness: (0.000, 1.000, 42.185)
[0s] generation 200 took 0ms, median: 0ms fitness: (0.000, 1.000, 42.185)
[0s] generation 300 took 0ms, median: 0ms fitness: (0.000, 1.000, 42.185)
[0s] generation 400 took 0ms, median: 0ms fitness: (0.000, 1.000, 42.185)
[

**NOTE**: if you have changed amount of jobs tasks and/or vehicle types and left matrix definition without necessary changes, then simply pass an empty array to `matrices` property to use an approximation fallback functionality. Otherwise, you will get an error as dimension of matrix won't match amount of unique locations in the problem definition.

As you can see, by default some logging about search progression is enabled. The function returns a `Solution`, serialized as json string. To get it as an typed object, we can simply run deseriazer:

In [11]:
solution = prg.Solution(**json.loads(solution_json))

solution

Solution(statistic=Statistic(cost=42.185199999999995, distance=13251, duration=3507, times=Times(driving=2367, serving=1140, waiting=0, commuting=0, parking=0)), tours=[Tour(vehicleId='vehicle_1', typeId='vehicle', shiftIndex=0, stops=[Stop(location=Location(lat=52.5316, lng=13.3884), time=Schedule(arrival=datetime.datetime(2024, 7, 4, 9, 0, tzinfo=datetime.timezone.utc), departure=datetime.datetime(2024, 7, 4, 9, 29, 3, tzinfo=datetime.timezone.utc)), distance=0, load=[1], activities=[Activity(jobId='departure', type='departure', location=None, time=None, jobTag=None)]), Stop(location=Location(lat=52.52599, lng=13.45413), time=Schedule(arrival=datetime.datetime(2024, 7, 4, 9, 44, 51, tzinfo=datetime.timezone.utc), departure=datetime.datetime(2024, 7, 4, 9, 49, 51, tzinfo=datetime.timezone.utc)), distance=5112, load=[0], activities=[Activity(jobId='delivery_job', type='delivery', location=None, time=None, jobTag=None)]), Stop(location=Location(lat=52.5225, lng=13.4095), time=Schedule(a

Essentially, a `Solution` entity is represented by list of tours and overall `Statistic`. Each tour has its own statistic and list of tours. More information about solution structure can be found [here](https://reinterpretcat.github.io/vrp/concepts/pragmatic/solution/index.html).